In [2]:
import threading

from odps import ODPS
from odps import options


def download(pt):
    ACCESS_ID = 'LTAI5t9uwJrh5eJ7Q5E37D1s'
    SECRET_ACCESS_KEY = 'NCFHOAnvqfnTrpypgR4b3cNawP8fnB'
    ODPS_PROJECT = 'OpenDigger_prod_dev'
    ODPS_ENDPOINT = 'http://service.cn-shanghai.maxcompute.aliyun.com/api'
    # 创建odps实例
    o = ODPS(ACCESS_ID, SECRET_ACCESS_KEY,
             project=ODPS_PROJECT, endpoint=ODPS_ENDPOINT)
    options.tunnel.limit_instance_tunnel = False
    sql = """
    SELECT DISTINCT actor_id, repo_id, repo_stargazers_count, repo_forks_count
    FROM ods_github_log
    WHERE  pt=%s
    AND type='PullRequestEvent'
    """ % pt
    filename = '.log%s.csv' % pt
    with open(filename, 'w') as f:
        f.write('actor_id,repo_id,repo_stargazers_count,repo_forks_count\n')
        with o.execute_sql(sql).open_reader() as reader:
            for i in reader:
                line = (
                        str(i.actor_id) + ','
                        + str(i.repo_id) + ','
                        + str(i.repo_stargazers_count) + ','
                        + str(i.repo_forks_count) + ',' + '\n')
                f.write(line)


class DownloadThread(threading.Thread):
    def __init__(self, pt):
        super(DownloadThread, self).__init__()
        self.pt = pt

    def run(self):
        download(pt)


if __name__ == '__main__':
    threads = []
    ptList = ['20210101', '20210201', '20210301', '20210401', '20210501', '20210601',
              '20210701', '20210801', '20210901', '20211001', '20211101', '20211201']
    for pt in ptList:
        thread = DownloadThread(pt)
        thread.start()
        threads.append(thread)
    for t in threads:
        t.join()

In [2]:
from select import select
from odps import ODPS
from odps import options
from odps.df import DataFrame
import pandas as pd
import numpy as np
import re
import math
from collections import Counter
from collections import defaultdict

ACCESS_ID = 'LTAI5t9uwJrh5eJ7Q5E37D1s'
SECRET_ACCESS_KEY = 'NCFHOAnvqfnTrpypgR4b3cNawP8fnB'
ODPS_PROJECT = 'OpenDigger_prod_dev'
ODPS_ENDPOINT = 'http://service.cn-shanghai.maxcompute.aliyun.com/api'
# ACCESS_ID = 'LTAI5tSjrYp6JDW2bbiTdegU'
# SECRET_ACCESS_KEY = 'OE6JbSqOZUey5fzr9Wg6fuYYKvslZx'
# 创建odps实例
o = ODPS(ACCESS_ID, SECRET_ACCESS_KEY,
         project=ODPS_PROJECT, endpoint=ODPS_ENDPOINT)
options.tunnel.limit_instance_tunnel = False
# options.read_timeout = 3600000

users = DataFrame(o.get_table('ods_github_users'))
print(users.dtypes)

github_repo = DataFrame(o.get_table('dim_github_actor'))
print(github_repo.dtypes)

odps.Schema {
  login                     string        
  created_at                date          
  database_id               int64         
  location                  string        
  company                   string        
  bio                       string        
  is_employee               boolean       
  email                     string        
  infoname                  string        
  followers                 string        
  following                 string        
  time                      date          
  name                      string        
  lastupdatedat             date          
  nextupdateat              date          
}

odps.Schema {
  actor_id              int64           
  actor_login           string          
  type                  string          
  created_at            datetime        
}



In [2]:
print(github_repo.head(3))

TransientProgressBar(value=0.0)

InstancesProgress()

HTMLNotifier()

   repo_id                     repo_name  org_id org_login repo_description  \
0    20798  svenfuchs/reference_tracking       0                              
1    22698                averyj/ramazon       0                              
2    25247               kings/alonetone       0                              

   repo_size  repo_stargazers_count  repo_forks_count repo_language  \
0          0                      0                 0                 
1          0                      0                 0                 
2          0                      0                 0                 

   repo_has_issues  repo_has_projects  repo_has_downloads  repo_has_wiki  \
0                0                  0                   0              0   
1                0                  0                   0              0   
2                0                  0                   0              0   

   repo_has_pages repo_license repo_default_branch repo_created_at  \
0               0      

In [16]:
def exe_sql(sql):
    data = []
    with o.execute_sql(sql).open_reader() as reader:

        d = defaultdict(list)  
        for record in reader:
            for res in record:
                d[res[0]].append(res[1])  
        data = pd.DataFrame.from_dict(d,orient='index').T 
    return data
rdata =  exe_sql("SELECT actor_id, repo_id, type, created_at FROM ods_github_log;")

ODPSError: ODPS-0130071: InstanceId: 2022051106241096g9q4os2t2
ODPS-0130071:[0,0] Semantic analysis exception - physical plan generation failed: java.lang.RuntimeException: Table(opendigger_prod_dev,ods_github_log) is full scan with all partitions, please specify partition predicates.



In [6]:
rdata

,repo_id,repo_name,repo_forks_count,repo_has_pr
0,20798,svenfuchs/reference_tracking,0,0
1,22698,averyj/ramazon,0,0
2,25247,kings/alonetone,0,0
3,36812,takatoshiono/chin-ma-ya.org,0,0
4,38382,dcrec1/in_place_editing_i18n,0,0
...,...,...,...,...
99995,211570557,direwolf-github/my-app-ee0b9076,0,1
99996,211570962,iggfoot/samjacksonquizapp,0,0
99997,211572654,devhub-blue-sea/org-public-empty-repo-us-west-2,0,0
99998,211573589,michaelmalonenz/craftinginterpreters,0,0
